In [1]:
import numpy as np
import pandas as pd
import time

import requests
import bs4
import json
import re

import tokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
def stock_prices(ticker):
    """
    ticker is the abbreviated symbol for a stock e.g.AAPL
    this function returns the daily price history of the requested stock as a dataframe
    """
    stock_endpoint = 'https://financialmodelingprep.com/api/v3/historical-price-full/'
    response = requests.get(stock_endpoint + ticker + '?apikey=70407133ea11d7284c70bbca4eee2547').json()
    type(response) == dict
    return pd.DataFrame(response['historical'])

def ny_times_articles(keyword):
    url_dict = {}
    index = 0
    for i in range(10):
        response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?q="+ keyword +"&fq=news_desk:Business&page="+str(i)+"&api-key=fO0tDSRQQdU68GkuXbMjt1uA2FYImzVp").json()
        try:
            docs = response['response']['docs']
            for item in docs:
                if item['web_url'][8] == 'w': # To remove 'https://bits.blogs.nytimes...' or ''https://dealbook.nytimes...'
                    link = item['web_url']
                    date = link[24:28] + '-' + link[29:31] + '-' + link[32:34]
                    url_dict[index] = tuple((link, date))
                    index += 1
        except KeyError:
            continue
    return url_dict

In [3]:
stocks = stock_prices('AAPL')
url_dict = ny_times_articles('apple')

In [4]:
# use the closing prices of the stock as the daily prices and the differences in each day as the percentage rise or fall
# the difference of stock prices between date1 and date2 will be stored in the row of date1 for ease of comparison

prices = stocks['close']
dates = stocks['date']
difference = stocks['close'].astype('float64').diff()
pct_change = difference / prices

stock = pd.DataFrame({'date': dates, 'price':prices, 'change': difference, 'pct_change': pct_change})
stock

,date,price,change,pct_change
0,2020-12-31,132.690002,NaN,NaN
1,2020-12-30,133.720001,1.029999,0.007703
2,2020-12-29,134.869995,1.149994,0.008527
3,2020-12-28,136.690002,1.820007,0.013315
4,2020-12-24,131.970001,-4.720001,-0.035766
...,...,...,...,...
1254,2016-01-08,24.240000,-0.392500,-0.016192
1255,2016-01-07,24.112499,-0.127501,-0.005288
1256,2016-01-06,25.174999,1.062500,0.042205
1257,2016-01-05,25.677500,0.502501,0.019570


In [5]:
tokenized = tokenizer.tokenize_sentence(url_dict)

In [6]:
merged = tokenizer.merged

In [7]:
tokenized_word = tokenizer.tokenizer_myself(tokenized)

In [8]:
sentence_vals = tokenizer.sentence_calculator(tokenized_word)

In [9]:
article_vals = tokenizer.calculate_vals(sentence_vals)

In [10]:
# Delete articles (given index from tokenize_sentence function)
deletable_index = tokenizer.get_deletable_index()
for index in deletable_index:
    del url_dict[index]

In [11]:
# Add article scores to dataframe

In [12]:
stock['score'] = 0
stock

,date,price,change,pct_change,score
0,2020-12-31,132.690002,NaN,NaN,0
1,2020-12-30,133.720001,1.029999,0.007703,0
2,2020-12-29,134.869995,1.149994,0.008527,0
3,2020-12-28,136.690002,1.820007,0.013315,0
4,2020-12-24,131.970001,-4.720001,-0.035766,0
...,...,...,...,...,...
1254,2016-01-08,24.240000,-0.392500,-0.016192,0
1255,2016-01-07,24.112499,-0.127501,-0.005288,0
1256,2016-01-06,25.174999,1.062500,0.042205,0
1257,2016-01-05,25.677500,0.502501,0.019570,0


In [13]:
for element in url_dict:
    print(url_dict[element][1])

2020-12-16
2020-12-13
2020-12-23
2020-12-17
2020-12-17
2020-12-01
2020-12-15
2020-12-14
2020-12-09
2020-11-18
2020-11-10
2020-11-09
2020-11-10
2020-10-27
2020-10-25
2020-10-29
2020-10-20
2020-10-22
2020-10-21
2020-10-09
2020-10-06
2020-10-13
2020-10-12
2020-10-08
2020-09-28
2020-10-06
2020-10-06
2020-09-24
2020-09-28
2020-09-17
2020-09-16
2020-09-01
2020-09-04
2020-08-20
2020-09-03
2020-08-25
2020-08-19
2020-08-14
2020-08-26
2020-08-13
2020-08-04
2020-08-19
2020-08-19
2020-08-18
2020-08-18
2020-08-07
2020-08-14
2020-07-28
2020-08-10
2020-07-28
2020-08-08
2020-07-15
2020-07-31
2020-07-31
2020-07-30
2020-07-30
2020-07-30
2020-07-30
2020-07-29
2020-07-29
2020-07-29
2020-07-29
2020-06-29
2020-07-28
2020-06-19
2020-06-18
2020-07-22
2020-06-22
2020-07-21
2020-07-20
2020-06-16
2012-07-26
2008-07-26
2020-07-15
2008-07-24
2015-02-20
2013-05-06
2012-02-27
2011-08-10
2010-10-18
2016-04-06
2013-02-09
2011-02-18
2014-08-09
2010-09-06
2010-06-24
2006-08-25
2019-06-27
2006-06-30


In [14]:
# Using article_val's elements (ordered by date given in url_dict, since tokenizer uses for loop with iteration)
remove_index = []
for index in url_dict:
    try:
        stock_index = stock[stock['date'] == url_dict[index][1]].index[0]
        stock.loc[stock_index, 'score'] = article_vals[index]
    except IndexError:
        remove_index.append(index)

stock[0:30]

,date,price,change,pct_change,score
0,2020-12-31,132.690002,NaN,NaN,0.000000
1,2020-12-30,133.720001,1.029999,0.007703,0.000000
2,2020-12-29,134.869995,1.149994,0.008527,0.000000
3,2020-12-28,136.690002,1.820007,0.013315,0.000000
4,2020-12-24,131.970001,-4.720001,-0.035766,0.000000
5,2020-12-23,130.960007,-1.009994,-0.007712,0.241962
6,2020-12-22,131.880005,0.919998,0.006976,0.000000
7,2020-12-21,128.229996,-3.650009,-0.028465,0.000000
8,2020-12-18,126.660004,-1.569992,-0.012395,0.000000
9,2020-12-17,128.699997,2.039993,0.015851,-0.192564


In [15]:
# Create ML using Linear Regression
from sklearn.model_selection import train_test_split # Data Splitting
from sklearn.linear_model import LinearRegression # LinearRegression Model


In [16]:
# Clean data
stock.dropna()

,date,price,change,pct_change,score
1,2020-12-30,133.720001,1.029999,0.007703,0.000000
2,2020-12-29,134.869995,1.149994,0.008527,0.000000
3,2020-12-28,136.690002,1.820007,0.013315,0.000000
4,2020-12-24,131.970001,-4.720001,-0.035766,0.000000
5,2020-12-23,130.960007,-1.009994,-0.007712,0.241962
...,...,...,...,...,...
1254,2016-01-08,24.240000,-0.392500,-0.016192,0.000000
1255,2016-01-07,24.112499,-0.127501,-0.005288,0.000000
1256,2016-01-06,25.174999,1.062500,0.042205,0.000000
1257,2016-01-05,25.677500,0.502501,0.019570,0.000000


In [17]:
# Change data's type to date
import datetime

string_date = stock['date'].to_list()
datetime_lst = []

for date in string_date:
    date_obj = datetime.datetime.strptime(date, '%Y-%m-%d')
    date_int = (10000 * date_obj.year) + (100 * date_obj.month) + (date_obj.day)
    datetime_lst.append(date_int)

stock['date'] = datetime_lst

stock

,date,price,change,pct_change,score
0,20201231,132.690002,NaN,NaN,0.0
1,20201230,133.720001,1.029999,0.007703,0.0
2,20201229,134.869995,1.149994,0.008527,0.0
3,20201228,136.690002,1.820007,0.013315,0.0
4,20201224,131.970001,-4.720001,-0.035766,0.0
...,...,...,...,...,...
1254,20160108,24.240000,-0.392500,-0.016192,0.0
1255,20160107,24.112499,-0.127501,-0.005288,0.0
1256,20160106,25.174999,1.062500,0.042205,0.0
1257,20160105,25.677500,0.502501,0.019570,0.0
